In [ ]:
# Common
import os 
import keras
import numpy as np
import tensorflow as tf

# Data 
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator 
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, StratifiedKFold

# Data Visualization 
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
# Model 
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dropout, Dense, GlobalAvgPool2D, Flatten, MaxPooling2D

# Optimizers
from keras.optimizers import SGD, Adam

# Callbacks 
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

### Data

Download from:

- https://www.nist.gov/srd/nist-special-database-19
- https://www.kaggle.com/datasets/sachinpatel21/az-handwritten-alphabets-in-csv-format/download?datasetVersionNumber=5
- https://data-flair.s3.ap-south-1.amazonaws.com/Data-Science-Code/handwritten-character-recognition-code.zip

In [ ]:
# Specify Data Path
file_path = 'A_Z Handwritten Data.csv'

# Column Names
names = ['class']
for id in range(1,785):
    names.append(id)

In [ ]:
%%time

# Load Data
df = pd.read_csv(file_path,header=None, names=names)
df.head()

In [ ]:
class_mapping = {}
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
for i in range(len(alphabets)):
    class_mapping[i] = alphabets[i]
class_mapping

In [ ]:
names = df['class'].value_counts().keys().map(class_mapping)
values = df['class'].value_counts()

### Statistics

In [ ]:
# Plot Class Distribution
fig = px.pie(
    names=names,
    values=values,
    height=800,
    title='Class Distribution'
)
fig.update_layout({'title':{'x':0.5}})
fig.show()

In [ ]:
# Plot Class Distribution
fig = px.bar(
    x=names,
    y=values,
    height=800,
    title='Class Distribution'
)
fig.update_layout({'title':{'x':0.5}})
fig.show()

### Data Loading

In [ ]:
def normalize(x):
    x = (x.astype('float32') - 127.5) / 127.5
    return x

def unnormalize(x):
    x = 127.5*x+127.5
    return x

In [ ]:
last_activation="sigmoid"

In [ ]:
if last_activation=="softmax":
    y_full_id = df.pop('class').to_numpy()
    y_full = keras.utils.to_categorical(y_full_id)
    
else:
    y_full = df.pop('class').to_numpy()

x_orig = df.to_numpy().reshape(-1,28,28, 1)
#x_full = normalize(x_orig) # -1 to 1
x_full = x_orig/255.0 # 0 to 1

In [ ]:
# x_full = (x_full-1.0)*(-1) # inverting

In [ ]:
x_full.max(), x_full.min()

In [ ]:
splitter = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
for train_ids, test_ids in splitter.split(x_full, y_full):
    X_train, y_train = x_full[train_ids], y_full[train_ids]
    X_test, y_test = x_full[test_ids], y_full[test_ids]

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(x_full, y_full, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, y_train.shape

### Data Visualization

In [ ]:
plt.figure(figsize=(15,8))
for i in range(1, 11):
    
    id_ = np.random.randint(len(X_train))
    if last_activation=="softmax":
        image, label = X_train[id_].reshape(28,28), class_mapping[int(y_train[id_].argmax())]
    else:
        image, label = X_train[id_].reshape(28,28), class_mapping[int(y_train[id_])]
    
    plt.subplot(2,5,i)
    plt.imshow(image, cmap='binary')
    plt.title(label)
    plt.axis('off')
    
plt.tight_layout()
plt.show()

### Model

In [ ]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(26, activation=last_activation))

In [ ]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
# model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(26, activation=last_activation))

In [ ]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
# model.add(BatchNormalization())
# model.add(Dense(26, activation=last_activation))

In [ ]:
# Model Architecture
model = Sequential([
    Conv2D(32, kernel_size=3, strides=2, padding='same', kernel_initializer='he_normal', input_shape=(28, 28, 1)),
    MaxPool2D(),

    BatchNormalization(),
    Conv2D(64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
    BatchNormalization(),
    Conv2D(64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
    MaxPool2D(),

    BatchNormalization(),
    Conv2D(128, kernel_size=3, padding='same', kernel_initializer='he_normal'),
    BatchNormalization(),
    Conv2D(128, kernel_size=3, padding='same', kernel_initializer='he_normal'),
    MaxPool2D(),

    BatchNormalization(),
    Conv2D(256, kernel_size=3, padding='same', kernel_initializer='he_normal'),
    BatchNormalization(),
    Conv2D(256, kernel_size=3, padding='same', kernel_initializer='he_normal'),
  
    GlobalAvgPool2D(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(26, activation=last_activation)
])

In [ ]:
model.summary()

### Training

In [ ]:
# Compile

opt = Adam(learning_rate=0.0001)

if last_activation=="softmax":
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )

else:
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )

In [ ]:
# Callbacks
cbs = [EarlyStopping(patience=8, restore_best_weights=True), ModelCheckpoint("model.h5", save_best_only=True)]

In [ ]:
history = model.fit(
            X_train, y_train,
            validation_split=0.25,
            epochs=50,
            batch_size=64,
            callbacks=cbs,
            verbose=1
        )

In [ ]:
print("The validation accuracy is :", history.history['val_accuracy'])
print("The training accuracy is :", history.history['accuracy'])
print("The validation loss is :", history.history['val_loss'])
print("The training loss is :", history.history['loss'])

### EValuating

In [ ]:
model = load_model('model.h5')

In [ ]:
model.evaluate(X_test,y_test)

### Predictions

In [ ]:
plt.figure(figsize=(20,20))
for i in range(1, 101):
    
    id_ = np.random.randint(len(X_test))
    
    if last_activation=="softmax":
        image, label = X_test[id_].reshape(28,28), class_mapping[int(y_test[id_].argmax())]
    else:
        image, label = X_test[id_].reshape(28,28), class_mapping[int(y_test[id_])]
    
    pred = class_mapping[int(np.argmax(model.predict(image.reshape(-1,28,28,1))))]
    
    plt.subplot(10,10,i)
    plt.imshow(image, cmap='binary')
    plt.title(f"Org: {label}, Pred: {pred}")
    plt.axis('off')
    
plt.tight_layout()
plt.show()